In [ ]:
import gc
gc.collect()

254

In [ ]:
import os
import cv2
import numpy as np
import csv
import shutil
import datetime
import time

# Extract Frames

In [ ]:
DATA_DIR = '/home/ycaun/Desktop/jeheng/Validate'
DATA2_DIR = '/home/ycaun/Desktop/jeheng/Validate2'
#CATEGORIES = ['dining', 'drawing', 'mopping_floor', 'reading_book', 'running_on_treadmill','sleeping','watching_tv']
CATEGORIES = ['sewing']
NUM_CLASSES = len(CATEGORIES)
SEQUENCE_LENGTH = 80

In [ ]:
def extract_frames(video_file, category):
    cap = cv2.VideoCapture(video_file)
    frames = []
    vid_name = os.path.splitext(os.path.basename(video_file))[0]
    frame_dir = os.path.join(DATA2_DIR, category, os.path.splitext(os.path.basename(video_file))[0])
    loaded = True

    with open(os.path.join(DATA2_DIR, 'Skip_vid_report.csv'), mode='a', newline='') as file1:
        with open(os.path.join(DATA2_DIR, 'Dataset_frames_report.csv'), mode='a', newline='') as file2:
            writer1 = csv.writer(file1)
            writer2 = csv.writer(file2)
            video_frames_count = int(cap.get(cv2.CAP_PROP_FRAME_COUNT))

            if video_frames_count < SEQUENCE_LENGTH:
                writer1.writerow([category, video_file, str(video_frames_count)])
                loaded = False
                return 0, loaded

            else:
                os.makedirs(frame_dir, exist_ok=True)
                writer2.writerow([category, video_file, str(video_frames_count)])

                skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH), 1)

                with open(os.path.join(DATA2_DIR, 'frames.csv'), mode='a', newline='') as file3:
                    writer3 = csv.writer(file3)

                    for frame_counter in range(SEQUENCE_LENGTH):

                        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)

                        success, frame = cap.read()

                        if not success:
                            print("Fail to read")
                            break

                        cv2.imwrite(os.path.join(frame_dir, f'{frame_counter}.jpg'), frame)
                        writer3.writerow([category, vid_name, os.path.join(frame_dir, f'{frame_counter}.jpg')])

                        frames.append(frame)

                    cap.release()

    return frames, loaded

In [ ]:
def preprocess_data(CATEGORIES):

    with open(os.path.join(DATA2_DIR, 'Extract_frame_report.csv'), mode='a', newline='') as file:
        writer3 = csv.writer(file)
        writer3.writerow(['Category', 'load_count', 'fail_count', 'load_error'])

        for i, category in enumerate(CATEGORIES):

            load_count = 0
            fail_count = 0
            load_error = 0
            category_dir = os.path.join(DATA_DIR, category)

            print('Loading ', category, ' : \n')
            start_time = datetime.datetime.now()
            print('Start at: ', start_time.strftime("%c"))

            for video_file in os.listdir(category_dir):
                video_path = os.path.join(category_dir, video_file)
                frames, loaded = extract_frames(video_path, category)
                if not loaded:
                    fail_count += 1
                    print('Fail to load ', video_file)
                else:
                    if len(frames) == SEQUENCE_LENGTH:
                        print('Succeed to load ', video_file)
                        load_count += 1
                        if load_count % 50 == 0:
                            gc.collect()
                            time.sleep(60)
                    else:
                        load_error += 1
                        print(video_file, ' Loaded with error')
            writer3.writerow([category, load_count, fail_count, load_error])
            end_time = datetime.datetime.now()
            print('End at: ', end_time.strftime("%c"))

            time_difference = (end_time - start_time).total_seconds() / 60
            print(f"Time taken: {time_difference:.2f} minutes")
            gc.collect()
            time.sleep(100)

    return 'Finish loading!'

In [ ]:
preprocess_data(CATEGORIES)

Loading  sewing  : 

Start at:  Sat Sep  9 03:47:41 2023
Succeed to load  JdqnS-vFELM_000003_000013.mp4
Succeed to load  JZD5c4hRC_o_000195_000205.mp4
Succeed to load  LdmwqjBEhx8_000015_000025.mp4
Succeed to load  kqURBmQKIG4_000042_000052.mp4
Succeed to load  J1KiP4YPjVc_000010_000020.mp4
Succeed to load  LpUp76MfbBU_000198_000208.mp4
Succeed to load  NAaPjikjmfA_000013_000023.mp4
Succeed to load  L27AMvA07co_000015_000025.mp4
Succeed to load  l3p9iVMtwDs_000057_000067.mp4
Succeed to load  mPJCxaOJnDI_000056_000066.mp4
Succeed to load  JT2SAPN95Mo_000050_000060.mp4
Succeed to load  kBuYxiNQrxI_000067_000077.mp4
Succeed to load  N4-_ISQs4u8_000214_000224.mp4
Succeed to load  LENIJzXoBmc_000083_000093.mp4
Succeed to load  HmU6tNe9wTM_000010_000020.mp4
Succeed to load  I1097PaV95c_000002_000012.mp4
Succeed to load  j0Y-UV14nIc_000020_000030.mp4
Succeed to load  IObq65IZCgk_000098_000108.mp4
Succeed to load  LcMc0Jp7lOY_000021_000031.mp4
Succeed to load  llnEugWEy_Y_000008_000018.mp4
Suc

'Finish loading!'

In [ ]:
gc.collect()